# Notebook facultatif : préparation du jeu de données à partir des données brutes

Quand on se lance avec enthousiasme dans un nouveau projet de machine-learning, on pense avant tout au choix du modèle que l'on va utiliser, ce qui constitue la partie "noble" de l'étude. 

Cependant, et même s'il ne s'agit pas de l'étape la plus agréable du travail, un préalable indispensable est de réunir les données brutes puis de les mettre en forme pour qu'elles puissent être ingérées par le modèle.

L'objectif de ce TP est d'effectuer ce travail de construction du jeux de données "propres" à partir de données brutes issues de sources diverses.

Nos fichiers d'entrée bruts sont les suivants :
> * historique de consommation nationale (source : Eco2mix) : **conso_2014_2018.csv**
> * calendrier des jours fériés : **jours_feries.csv**
> * coordonnées géographiques et les poids associés aux stations météo France : **stations_meteo_RTE.csv**
> * historique d'observations et de prévisions à 24h de température : **temperatures.csv**

Et les fichiers que l'on va créer sont :
> * **x_input_full.csv**  # Les entrées pour le modèle d'apprentissage
> * **x_input_light.csv**  # Une version allégée de *x_input_full*
> * **y_conso.csv**  # les sorties pour le modèle d'apprentissage

## Mise en place de l'environnement

Exécutez la cellule ci-dessous, par exemple avec shift-entrée.

In [1]:
# Chargement des bibliothèques python, et quelques éléments de configuration. 
# Si vous exécuter ce notebook depuis votre PC, il faudra peut-etre installer certaines librairies avec   
# "pip3 install ma_bibliotheque"
import os  # accès aux commandes système
import datetime  # structure de données pour gérer des objets calendaires
import pandas as pd  # gérer des tables de données en python
import numpy as np  # librairie d'opérations mathématiques
import zipfile  # manipulation de fichiers zip
from sklearn.preprocessing import StandardScaler  # normalisation des données
import joblib  # sauvegarde du scaler

In [2]:
data_folder = os.path.join(os.getcwd(), "data")
raw_data_folder = os.path.join(data_folder, "raw_data")
transformed_data_folder = os.path.join(data_folder, "transformed_data")

print("")
print("Mon repertoire de raw_data_folder est : {}".format(raw_data_folder))
print("")
print("Fichiers/dossiers contenus dans ce repertoire :")
for file in os.listdir(raw_data_folder):
    print(" - " + file)    


Mon repertoire de raw_data_folder est : /home/clementrem/Documents/Formation_FIFA/TP_reworked/data/raw_data

Fichiers/dossiers contenus dans ce repertoire :
 - stations_meteo_RTE.csv
 - conso_2014_2018.csv
 - jours_feries.csv
 - temperatures.zip


## import en mémoire des données

### Données de consommation

Dans un premier temps on importe les données de consommation réalisée à partir du fichier **conso_2014_2018.csv**.   
La date et l'heure sont données dans la première colonne, la consommation nationale dans la suivante.

In [3]:
# Les données du csv sont importées dans un objet de type DataFrame
conso_csv = os.path.join(raw_data_folder, "conso_2014_2018.csv")
conso_df = pd.read_csv(conso_csv, sep=",")

Vérifions que les données ont été importées correctement :

In [4]:
# Afficher les dimensions et le noms des colonnes de la data frame
print(conso_df.shape)  # Nombre de lignes, nombre de colonnes
display(conso_df.head(2))
display(conso_df.tail(2))

(43824, 2)


,ds,conso_real
0,2014-01-01T00:00:00Z,64660
1,2014-01-01T01:00:00Z,61362


,ds,conso_real
43822,2018-12-31T22:00:00Z,59901
43823,2018-12-31T23:00:00Z,63977


#### Petit détour pour gérer les dates

Le fichier **conso_2014_2018.csv** contient en particulier la colonnes **ds**. Celle-ci contient une variable de type *string* correspondant à la date et à l'heure. 

<img src="pictures/clock.png" width=60 height=60>

Pour manipuler facilement ces variables, il est recommandé de les convertir en objet de type **datetime**.

In [5]:
conso_df['ds'] = pd.to_datetime(conso_df['ds'], utc=True)

In [6]:
display(conso_df.head(2))

,ds,conso_real
0,2014-01-01 00:00:00+00:00,64660
1,2014-01-01 01:00:00+00:00,61362


La cellule ci-dessous a pour but d'illustrer comment utiliser ces objets **datetime**.  
Ces quelques lignes de code pourraient vous reservir ultérieurement.

In [7]:
# datetime vers string
noel_2017_date = datetime.date(2017, 12, 25)
noel_2017_str = datetime.datetime.strftime(noel_2017_date, format="%Y-%m-%d")
print("noel_2017_date vaut : {}, et est de type {}".format(noel_2017_date, str(type(noel_2017_date))))
print("noel_2017_str vaut : {}, et est de type {}".format(noel_2017_str, str(type(noel_2017_str))))
print("---")

# string vers datetime
starwars_day_2017_str = "2017-05-04"
starwars_day_2017_date = datetime.datetime.strptime(starwars_day_2017_str, "%Y-%m-%d")
print("starwars_day_2017_date vaut : {}, et est de type {}".format(starwars_day_2017_date, str(type(starwars_day_2017_date))))
print("D'ailleurs, c'était le " + str(starwars_day_2017_date.weekday() + 1) + " ème jour de la semaine, où 0 correspond à lundi et 6 correspond à dimanche")
print("starwars_day_2017_str vaut : {}, et est de type {}".format(starwars_day_2017_str, str(type(starwars_day_2017_str))))
print("---")

# Voyager dans le temps
saint_sylvestre_2017_date = datetime.date(2017, 12, 31)
bienvenu_en_2018_date = saint_sylvestre_2017_date + datetime.timedelta(days=1)
print("Le 31 décembre 2017 plus un jour ça donne le {}".format(bienvenu_en_2018_date))

noel_2017_date vaut : 2017-12-25, et est de type <class 'datetime.date'>
noel_2017_str vaut : 2017-12-25, et est de type <class 'str'>
---
starwars_day_2017_date vaut : 2017-05-04 00:00:00, et est de type <class 'datetime.datetime'>
D'ailleurs, c'était le 4 ème jour de la semaine, où 0 correspond à lundi et 6 correspond à dimanche
starwars_day_2017_str vaut : 2017-05-04, et est de type <class 'str'>
---
Le 31 décembre 2017 plus un jour ça donne le 2018-01-01


### Récuperation des jours fériés

Le fichier **jours_feries.csv** contient la liste des jours fériés. 

In [8]:
jours_feries_csv = os.path.join(raw_data_folder,"jours_feries.csv")
jours_feries_df = pd.read_csv(jours_feries_csv, sep=";")

jours_feries_df.ds = pd.to_datetime(jours_feries_df.ds, utc=True)  # comme ci-dessus

jours_feries_df.head(3)

,ds,holiday
0,2012-12-25 00:00:00+00:00,Noel
1,2013-01-01 00:00:00+00:00,NouvelAn
2,2013-04-01 00:00:00+00:00,Paques


### Récupération des coordonnées géographiques des stations météo

Dans le but d'obtenir un modèle de machine-learning performant, nous allons créer une variable artificielle qui sera représentative de la température nationale France. Cette variable n'aura été relevée par aucun thermomètre, mais sera un barycentre des températures mesurées aux différentes stations météo. Chaque station météo s'est vue attribuer un poids permettant de sentir sa contribution. Pour en savoir plus sur les poids :  
https://clients.rte-france.com/lang/fr/visiteurs/services/actualites.jsp?id=9482&mode=detail

Commençons par charger le csv qui à chaque station météo attribue sa longitude/latitude/poids.

In [9]:
stations_meteo_csv = os.path.join(raw_data_folder, "stations_meteo_RTE.csv")
stations_meteo_df = pd.read_csv(stations_meteo_csv, sep=";")

In [10]:
stations_meteo_df.head(3)

,ID,Nom,longitude,latitude,Poids
0,2,BOULOGNE-SUR-MER,1.61667,50.71667,0.01
1,5,ABBEVILLE,1.83333,50.10000,0.01
2,15,LILLE,3.05858,50.63297,0.03


In [11]:
# Pour compter le nombre de stations il suffit de compter le nombre de lignes dans le data-frame
nb_stations = stations_meteo_df.shape[0]
print(nb_stations)

35


In [12]:
# Verifions que la somme des poids fait 1
np.sum(stations_meteo_df["Poids"])

1.0000000000000002

### Récupération des températures

<img src="pictures/weather.png" width=60 height=60>

On va utiliser les mêmes fonctions que précédemment pour lire le fichier **temperatures.csv**, qui contient les historiques des température réalisées et prévues pour différentes stations météo France.

**Attention : Les données météo sont encryptées dans un fichier zip.**  
Pour les lire vous avez besoin d'un mot de passe qui ne peut vous être donné que dans le cadre d'un travail au sein de RTE.

In [13]:
meteo_zip = os.path.join(raw_data_folder, "temperatures.zip")

In [14]:
password = "FIFA_Meteo"

In [15]:
# Cette étape peut être un peu longue car le fichier est volumineux
zip_file_meteo = zipfile.ZipFile(meteo_zip)
zip_file_meteo.setpassword(bytes(password, 'utf-8'))
meteo_df = pd.read_csv(zip_file_meteo.open('temperatures.csv'), sep=",", engine='c', header=0)
meteo_df['ds'] = pd.to_datetime(meteo_df.date_cible, utc=True)
meteo_df = meteo_df.drop(columns=['date_cible'])

In [16]:
print(meteo_df.shape)  # (nb lignes , nb_colonnes)
print(meteo_df.columns)

(43577, 71)
Index(['002_0', '002_24', '005_0', '005_24', '015_0', '015_24', '027_0',
       '027_24', '070_0', '070_24', '110_0', '110_24', '120_0', '120_24',
       '130_0', '130_24', '145_0', '145_24', '149_0', '149_24', '156_0',
       '156_24', '168_0', '168_24', '180_0', '180_24', '190_0', '190_24',
       '222_0', '222_24', '240_0', '240_24', '255_0', '255_24', '260_0',
       '260_24', '280_0', '280_24', '299_0', '299_24', '434_0', '434_24',
       '460_0', '460_24', '481_0', '481_24', '497_0', '497_24', '510_0',
       '510_24', '579_0', '579_24', '588_0', '588_24', '621_0', '621_24',
       '630_0', '630_24', '643_0', '643_24', '645_0', '645_24', '650_0',
       '650_24', '675_0', '675_24', '690_0', '690_24', '747_0', '747_24',
       'ds'],
      dtype='object')


In [17]:
display(meteo_df.head(3))
display(meteo_df.tail(3))

,002_0,002_24,005_0,005_24,015_0,015_24,027_0,027_24,070_0,070_24,...,645_24,650_0,650_24,675_0,675_24,690_0,690_24,747_0,747_24,ds
0,7.2,7.4,5.5,6.4,5.9,5.4,5.5,6.6,6.6,5.7,...,7.1,9.60,7.3,0.5,3.4,5.6,6.5,4.2,7.6,2013-12-31 23:00:00+00:00
1,7.1,7.1,5.5,6.2,6.5,5.2,5.5,6.6,6.8,5.8,...,6.6,9.69,7.1,1.0,2.8,5.6,6.4,3.6,7.7,2014-01-01 00:00:00+00:00
2,6.9,6.9,5.2,6.0,5.4,5.2,6.0,6.6,6.8,5.7,...,6.5,8.60,6.6,0.4,2.4,5.5,6.2,3.5,7.5,2014-01-01 01:00:00+00:00


,002_0,002_24,005_0,005_24,015_0,015_24,027_0,027_24,070_0,070_24,...,645_24,650_0,650_24,675_0,675_24,690_0,690_24,747_0,747_24,ds
43574,NaN,11.6,NaN,12.8,NaN,13.4,NaN,13.5,NaN,12.9,...,11.2,NaN,14.0,NaN,13.6,NaN,13.8,NaN,13.9,2018-12-21 13:00:00+00:00
43575,NaN,11.2,NaN,12.3,NaN,13.1,NaN,13.3,NaN,13.0,...,10.8,NaN,13.8,NaN,13.6,NaN,13.8,NaN,13.9,2018-12-21 14:00:00+00:00
43576,NaN,11.2,NaN,11.8,NaN,12.7,NaN,13.0,NaN,12.9,...,10.0,NaN,13.2,NaN,13.0,NaN,13.2,NaN,13.1,2018-12-21 15:00:00+00:00


On remarque que les données s'arrête au 21/12/2018 et que pour cette date, on ne dispose que des prévisions à 24h, pas des réalisations.

### Une variable complémentaire à considérer : les données TEMPO

Pour l'exemple de la prévision de consommation, il serait pertinent de fournir en entrée du modèle l'information sur le type de jour Tempo. Les clients ayant souscrit à ce type de contrat sont incités à réduire leur consommations les jours BLANC et ROUGE, aussi on peut penser que cette information permettra d'améliorer la qualité des prédictions.

Elles ne sont pas utilisées dans le cadre de ce TP, mais le stagiaire motivé pourra trouver des informations complémentaires sur le web :  
http://www.rte-france.com/fr/eco2mix/eco2mix-telechargement

##  Fusion des données

<img src="pictures/fusion.png" width=600 height=200>

On va maintenant construire un dataframe unique qui regroupe toutes les données nécessaire à notre modèle de prévision. On aura ici une ligne pour chaque timestamp, et dans cette ligne à la fois notre X et notre Y pour le futur modèle de machine-learning.

### Fusion des données de consommation et de jours fériés

**Attention**  
L'heure n'apparaît pas dans *jours_feries_df*, mais implicitement, elle vaut 00:00 pour toutes les observations.  
Il faut y prendre garde lors de la la fusion avec les données de consommation, afin que le point horaire de 15h35 d'un jour férié soit bien reconnu comme appartenant à un jour férié. Pour cela, nous allons créer dans les deux jeux de données une colonne **day**, qui ne conserve que la date. Nous utiliserons ensuite cette colonne pour fusionner les deux jeux de données.

In [18]:
# On rajoute la colonne *day* aux 2 dataframes
conso_df['day'] = conso_df['ds'].apply(lambda x: datetime.datetime.strftime(x, format="%Y-%m-%d"))
jours_feries_df['day'] = jours_feries_df['ds'].apply(lambda x: datetime.datetime.strftime(x, format="%Y-%m-%d"))

# On merge sur cette colonne
merged_df = pd.merge(conso_df, jours_feries_df, on='day', how="left", suffixes=("", "_tmp"))

# On peut maintenant supprimer ces colonnes temporaires
merged_df = merged_df[["ds", "conso_real", "holiday"]]
display(merged_df.loc[0:24])

,ds,conso_real,holiday
0,2014-01-01 00:00:00+00:00,64660,NouvelAn
1,2014-01-01 01:00:00+00:00,61362,NouvelAn
2,2014-01-01 02:00:00+00:00,60748,NouvelAn
3,2014-01-01 03:00:00+00:00,58061,NouvelAn
4,2014-01-01 04:00:00+00:00,54475,NouvelAn
5,2014-01-01 05:00:00+00:00,52534,NouvelAn
6,2014-01-01 06:00:00+00:00,52262,NouvelAn
7,2014-01-01 07:00:00+00:00,52302,NouvelAn
8,2014-01-01 08:00:00+00:00,52429,NouvelAn
9,2014-01-01 09:00:00+00:00,51663,NouvelAn


In [19]:
merged_df["is_bank_holiday"] = merged_df["holiday"].astype("str").apply(lambda x: 0 if x == "nan" else 1)
merged_df.drop(['holiday'], axis=1, inplace=True)

# Regardons
display(merged_df.loc[0:24])

,ds,conso_real,is_bank_holiday
0,2014-01-01 00:00:00+00:00,64660,1
1,2014-01-01 01:00:00+00:00,61362,1
2,2014-01-01 02:00:00+00:00,60748,1
3,2014-01-01 03:00:00+00:00,58061,1
4,2014-01-01 04:00:00+00:00,54475,1
5,2014-01-01 05:00:00+00:00,52534,1
6,2014-01-01 06:00:00+00:00,52262,1
7,2014-01-01 07:00:00+00:00,52302,1
8,2014-01-01 08:00:00+00:00,52429,1
9,2014-01-01 09:00:00+00:00,51663,1


### Ajout des informations de température.

In [20]:
merged_df = pd.merge(merged_df, meteo_df, on = 'ds')

In [21]:
print(merged_df.shape)
print(merged_df.columns)

(43576, 73)
Index(['ds', 'conso_real', 'is_bank_holiday', '002_0', '002_24', '005_0',
       '005_24', '015_0', '015_24', '027_0', '027_24', '070_0', '070_24',
       '110_0', '110_24', '120_0', '120_24', '130_0', '130_24', '145_0',
       '145_24', '149_0', '149_24', '156_0', '156_24', '168_0', '168_24',
       '180_0', '180_24', '190_0', '190_24', '222_0', '222_24', '240_0',
       '240_24', '255_0', '255_24', '260_0', '260_24', '280_0', '280_24',
       '299_0', '299_24', '434_0', '434_24', '460_0', '460_24', '481_0',
       '481_24', '497_0', '497_24', '510_0', '510_24', '579_0', '579_24',
       '588_0', '588_24', '621_0', '621_24', '630_0', '630_24', '643_0',
       '643_24', '645_0', '645_24', '650_0', '650_24', '675_0', '675_24',
       '690_0', '690_24', '747_0', '747_24'],
      dtype='object')


### Calcul de la variable artificielle "température France"

On va ajouter une colonne à notre dataframe, colonne que - par expérience/expertise - on sait pouvoir être utile pour prévoir la consommation.

La température France est une moyenne pondérée de la température de 32 stations. C'est ici que nous avons besoin des poids de *stations_meteo_df*.

In [22]:
merged_df['temperature_real'] = np.dot(merged_df[list(merged_df.columns[merged_df.columns.str.endswith("_0")])], stations_meteo_df['Poids'])
merged_df['temperature_prevue'] = np.dot(merged_df[list(merged_df.columns[merged_df.columns.str.endswith("_24")])], stations_meteo_df['Poids'])

In [23]:
print(merged_df.shape)
print(merged_df.columns)
display(merged_df.head(2))

(43576, 75)
Index(['ds', 'conso_real', 'is_bank_holiday', '002_0', '002_24', '005_0',
       '005_24', '015_0', '015_24', '027_0', '027_24', '070_0', '070_24',
       '110_0', '110_24', '120_0', '120_24', '130_0', '130_24', '145_0',
       '145_24', '149_0', '149_24', '156_0', '156_24', '168_0', '168_24',
       '180_0', '180_24', '190_0', '190_24', '222_0', '222_24', '240_0',
       '240_24', '255_0', '255_24', '260_0', '260_24', '280_0', '280_24',
       '299_0', '299_24', '434_0', '434_24', '460_0', '460_24', '481_0',
       '481_24', '497_0', '497_24', '510_0', '510_24', '579_0', '579_24',
       '588_0', '588_24', '621_0', '621_24', '630_0', '630_24', '643_0',
       '643_24', '645_0', '645_24', '650_0', '650_24', '675_0', '675_24',
       '690_0', '690_24', '747_0', '747_24', 'temperature_real',
       'temperature_prevue'],
      dtype='object')


,ds,conso_real,is_bank_holiday,002_0,002_24,005_0,005_24,015_0,015_24,027_0,...,650_0,650_24,675_0,675_24,690_0,690_24,747_0,747_24,temperature_real,temperature_prevue
0,2014-01-01 00:00:00+00:00,64660,1,7.1,7.1,5.5,6.2,6.5,5.2,5.5,...,9.69,7.1,1.0,2.8,5.6,6.4,3.6,7.7,6.46539,5.99449
1,2014-01-01 01:00:00+00:00,61362,1,6.9,6.9,5.2,6.0,5.4,5.2,6.0,...,8.60,6.6,0.4,2.4,5.5,6.2,3.5,7.5,6.33415,5.93226


### Ajout des variables retardées

Le modèle prédictif n'aura pas le droit d'avoir accès à la température réalisée du futur (dommage, ça rendrait notre métier plus facile :-)), mais en revanche nous avons accès à la connaissance de la température réalisée 24 heures avant.

De même pour la consommation.

Ainsi, nous avons accès au réalisé (température, conso nationale) du J-1, et nous avons une température prévue pour le point horaire cible. Charge à notre modéle de prévision de nous proposer une valeur d ela consommation nationale pour le point horaire cible.

In [24]:
merged_df["temperature_real_24h_avant"] = merged_df["temperature_real"].shift(24)

In [25]:
merged_df["conso_real_24h_avant"] = merged_df["conso_real"].shift(24)

<font color='blue'>

**Prenons quelques instants pour comprendre ces histoires de variables retardées.**

Imaginons que l'on veuille prédire la consommation nationale pour le point horaire cible du 09/01/2014 à 00h00 (ligne 24 de working_df)

Utiliser dans un modèle de prédiction :
* la consommation réalisée de la veille (08/01/2014 à 00h00)
* les infos de météo réalisée de la veille
* les **prévisions** météo du point horaire cible (09/01/2014 à 00h00)

semble être pertinent.

</font>
<br/>
  
<font color='green'>

* Est-ce que le shift a été fait dans le bon sens ?

</font>

## Supression des variables inutiles

In [26]:
merged_df = merged_df.drop("temperature_real", axis=1)
merged_df = merged_df.drop([x for x in merged_df.columns if (x.endswith("_0") or x.endswith("_24"))], axis=1)

print(merged_df.shape)
print(merged_df.columns)
display(merged_df.head(2))

(43576, 6)
Index(['ds', 'conso_real', 'is_bank_holiday', 'temperature_prevue',
       'temperature_real_24h_avant', 'conso_real_24h_avant'],
      dtype='object')


,ds,conso_real,is_bank_holiday,temperature_prevue,temperature_real_24h_avant,conso_real_24h_avant
0,2014-01-01 00:00:00+00:00,64660,1,5.99449,NaN,NaN
1,2014-01-01 01:00:00+00:00,61362,1,5.93226,NaN,NaN


## Suppression des NaN

On souhaite avoir un jeu de données complet : pas de trous dans les dates et pour chaque point horaire, toutes les informations disponibles (consommation, température réalisée et prévue).

On affiche donc les dates pour lesquelles il manque des informations et on conservera un intervalle ne contenant pas ces dates.

In [27]:
mask = merged_df.isnull().any(axis=1)

print(np.sum(mask))
print(merged_df.index[mask].tolist())

25
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 165]


En début de dataset, avec nos manipulation pour obtenir les valeurs de la veille, il est normal que l'on ait quelques *NaN* pour les colonnes 24h_avant.

Quant à la ligne d'index 165, bah... C'est la vie.

maintenant nous n'allons garder que les lignes sans *NaN*.

In [28]:
merged_df = merged_df[~mask]
merged_df.reset_index(drop=True, inplace=True)  # on demande de remettre à jour les numéros d'index de la DF pour commencer à 0

print(merged_df.shape)
print(merged_df.columns)
display(merged_df.head(2))

(43551, 6)
Index(['ds', 'conso_real', 'is_bank_holiday', 'temperature_prevue',
       'temperature_real_24h_avant', 'conso_real_24h_avant'],
      dtype='object')


,ds,conso_real,is_bank_holiday,temperature_prevue,temperature_real_24h_avant,conso_real_24h_avant
0,2014-01-02 00:00:00+00:00,59416,0,8.850575,6.46539,64660.0
1,2014-01-02 01:00:00+00:00,54881,0,8.805705,6.33415,61362.0


### Normalisation ("scaling")

In [29]:
columns_to_scale = ["conso_real", "temperature_prevue", "temperature_real_24h_avant", "conso_real_24h_avant"]
new_column_names = [x + "_scaled" for x in columns_to_scale]

scaler = StandardScaler()
scaler.fit(merged_df[columns_to_scale])
scaled_values = scaler.transform(merged_df[columns_to_scale])
scaled_values_df = pd.DataFrame(scaled_values, columns=new_column_names)

In [30]:
# Remarque en passant:
# Ne pas relancer cette cellule, sinon
# working_df va beaucoup grossir...
merged_df = pd.concat([merged_df, scaled_values_df], axis=1)

In [31]:
display(merged_df.head(2))
print(merged_df.shape)

# Petite verif manuelle
mean = np.mean(merged_df["conso_real"])
mean_scaled = np.mean(merged_df["conso_real_scaled"])
print()
print("--- Petite verif ---")
print("Moyenne de conso_real: " + str(mean))
print("Moyenne de conso_real_scaled: " + str(mean_scaled))

,ds,conso_real,is_bank_holiday,temperature_prevue,temperature_real_24h_avant,conso_real_24h_avant,conso_real_scaled,temperature_prevue_scaled,temperature_real_24h_avant_scaled,conso_real_24h_avant_scaled
0,2014-01-02 00:00:00+00:00,59416,0,8.850575,6.46539,64660.0,0.453969,-0.615538,-0.968581,0.897874
1,2014-01-02 01:00:00+00:00,54881,0,8.805705,6.33415,61362.0,0.070665,-0.622207,-0.988059,0.619043


(43551, 10)

--- Petite verif ---
Moyenne de conso_real: 54044.931712245416
Moyenne de conso_real_scaled: -2.1405526149049424e-16


In [32]:
# Pour plus tard, on se garde de côté un scaler uniquement dénormaliser la prédiction de conso
scaler_conso_nat = StandardScaler()
scaler_conso_nat.fit(merged_df[["conso_real"]])

# Verif
scaler_conso_nat.inverse_transform(merged_df["conso_real_scaled"])

array([59416., 54881., 54007., ..., 69176., 66831., 64814.])

### Encoding de la date

In [33]:
merged_df['month'] = merged_df['ds'].dt.month
merged_df['hour'] = merged_df['ds'].dt.hour

# On sépare les jours de la semaine en week-end / pas week-end
# De base, la fonction datetime.weekday() renvoie 0 => Lundi, 2 => Mardi, ..., 5 => Samedi, 6 => Dimanche
# Ci-dessous, si on a un jour de la semaine alors dans la colonne weekday on mettra 1, et 0 si c'est le week-end
merged_df['weekday'] = (merged_df['ds'].dt.weekday < 5).astype(int)  # conversion bool => int

# Regardons
merged_df.loc[[0, 1, 23, 24, 24 *7 -1, 24 * 7, 1100], :]

,ds,conso_real,is_bank_holiday,temperature_prevue,temperature_real_24h_avant,conso_real_24h_avant,conso_real_scaled,temperature_prevue_scaled,temperature_real_24h_avant_scaled,conso_real_24h_avant_scaled,month,hour,weekday
0,2014-01-02 00:00:00+00:00,59416,0,8.850575,6.465390,64660.0,0.453969,-0.615538,-0.968581,0.897874,1,0,1
1,2014-01-02 01:00:00+00:00,54881,0,8.805705,6.334150,61362.0,0.070665,-0.622207,-0.988059,0.619043,1,1,1
23,2014-01-02 23:00:00+00:00,63351,0,8.666080,9.699590,60626.0,0.786560,-0.642958,-0.488571,0.556818,1,23,1
24,2014-01-03 00:00:00+00:00,62135,0,8.685230,9.754945,59416.0,0.683782,-0.640112,-0.480356,0.454517,1,0,1
167,2014-01-09 00:00:00+00:00,61752,0,9.898510,10.467205,62008.0,0.651410,-0.459791,-0.374644,0.673660,1,0,1
168,2014-01-09 01:00:00+00:00,57162,0,9.818325,10.349260,57298.0,0.263458,-0.471708,-0.392149,0.275450,1,1,1
1100,2014-02-16 21:00:00+00:00,62438,0,6.169810,7.395080,62235.0,0.709392,-1.013961,-0.830599,0.692851,2,21,0


Nous allons maintenant effectuer un **"one-hot encoding"** pour convertir ces informations sous un format utilisable par un réseau de neurones.

In [34]:
encoded_month = pd.get_dummies(merged_df['month'], prefix="month")
encoded_hour = pd.get_dummies(merged_df['hour'], prefix="hour")

display(encoded_month.head(3))
display(encoded_hour.head(3))

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0


,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# Ne calculer cette cellule qu'une seule fois à cause du concat !
merged_df = pd.concat([merged_df, encoded_month, encoded_hour], axis=1)
merged_df = merged_df.drop(['month', 'hour'], axis=1)

In [36]:
print(merged_df.shape)
print(merged_df.columns)
display(merged_df.head(2))

(43551, 47)
Index(['ds', 'conso_real', 'is_bank_holiday', 'temperature_prevue',
       'temperature_real_24h_avant', 'conso_real_24h_avant',
       'conso_real_scaled', 'temperature_prevue_scaled',
       'temperature_real_24h_avant_scaled', 'conso_real_24h_avant_scaled',
       'weekday', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23'],
      dtype='object')


,ds,conso_real,is_bank_holiday,temperature_prevue,temperature_real_24h_avant,conso_real_24h_avant,conso_real_scaled,temperature_prevue_scaled,temperature_real_24h_avant_scaled,conso_real_24h_avant_scaled,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,2014-01-02 00:00:00+00:00,59416,0,8.850575,6.46539,64660.0,0.453969,-0.615538,-0.968581,0.897874,...,0,0,0,0,0,0,0,0,0,0
1,2014-01-02 01:00:00+00:00,54881,0,8.805705,6.33415,61362.0,0.070665,-0.622207,-0.988059,0.619043,...,0,0,0,0,0,0,0,0,0,0


## Sauvegarde du fichier 

Tout d'abord on sépare les données en deux : 
- le vecteur de consommation à prévoir : y_conso
- La matrice des variables explicatives : x_input

Sachant que plus tard notre modèle aura pour mission d'établir une correspondance _f_ telle que l'on ait du mieux possible une relation *y = f(X)*.

In [37]:
y_columns = ["ds", "conso_real", "conso_real_scaled"]

x_columns = ["ds", "is_bank_holiday"] \
+ ["temperature_real_24h_avant", 'temperature_real_24h_avant_scaled'] \
+ ["temperature_prevue", 'temperature_prevue_scaled'] \
+ ["conso_real_24h_avant", 'conso_real_24h_avant_scaled'] \
+ ["month_" + str(x) for x in range(1, 13)] \
+ ["hour_" + str(x) for x in range(0, 24)] \
+ ["weekday"]

In [38]:
y = merged_df[y_columns]

In [39]:
x = merged_df[x_columns]

In [40]:
x.to_csv(os.path.join(transformed_data_folder,"x.csv"), index=False, sep=";")
y.to_csv(os.path.join(transformed_data_folder,"y.csv"), index=False, sep=";")

In [41]:
joblib.dump(scaler_conso_nat, os.path.join(transformed_data_folder, "scaler_conso.save"))

['/home/clementrem/Documents/Formation_FIFA/TP_reworked/data/transformed_data/scaler_conso.save']

Et enfin on zip **x.csv** avec un mot de passe car ce fichier contient les infos de température.

Depuis un terminal :
> zip -e x.zip x.csv  
> rm x.csv